In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import random
import pickle

In [2]:
#importing various parts of the data below. print the columns as well.

In [3]:
ordered_items = pd.read_csv("~/Desktop/Desktop - Eric’s MacBook Pro/Git/Sentiment-analysis-and-Predicting-customer-satisfaction/Data/olist_order_items_dataset.csv")
ordered_items.columns

Index(['order_id', 'order_item_id', 'product_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value'],
      dtype='object')

In [4]:
payments = pd.read_csv("~/Desktop/Desktop - Eric’s MacBook Pro/Git/Sentiment-analysis-and-Predicting-customer-satisfaction/Data/olist_order_payments_dataset.csv")
payments.columns

Index(['order_id', 'payment_sequential', 'payment_type',
       'payment_installments', 'payment_value'],
      dtype='object')

In [5]:
review_scores = pd.read_csv("~/Desktop/Desktop - Eric’s MacBook Pro/Git/Sentiment-analysis-and-Predicting-customer-satisfaction/Data/olist_order_reviews_dataset.csv")
review_scores.columns

Index(['review_id', 'order_id', 'review_score', 'review_comment_title',
       'review_comment_message', 'review_creation_date',
       'review_answer_timestamp'],
      dtype='object')

In [6]:
delivery_info = pd.read_csv("~/Desktop/Desktop - Eric’s MacBook Pro/Git/Sentiment-analysis-and-Predicting-customer-satisfaction/Data//olist_orders_dataset.csv",
                           parse_dates =['order_purchase_timestamp','order_approved_at','order_delivered_carrier_date',
                                        'order_delivered_customer_date','order_estimated_delivery_date'])
delivery_info.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date'],
      dtype='object')

In [7]:
product_info = pd.read_csv("~/Desktop/Desktop - Eric’s MacBook Pro/Git/Sentiment-analysis-and-Predicting-customer-satisfaction/Data//olist_products_dataset.csv")
product_info.columns

Index(['product_id', 'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm'],
      dtype='object')

In [8]:
product_category_translation = pd.read_csv("~/Desktop/Desktop - Eric’s MacBook Pro/Git/Sentiment-analysis-and-Predicting-customer-satisfaction/Data//product_category_name_translation.csv")
product_category_translation.columns

Index(['product_category_name', 'product_category_name_english'], dtype='object')

In [9]:
#need to replace portugese product names with english. 
#the product_category_translation dataframe has the translations

In [10]:
df_product_dict = product_category_translation.to_dict()
df_product_dict
portugese_prod = df_product_dict['product_category_name']
english_prod = df_product_dict['product_category_name_english']

In [11]:
#using the dictionaries directly above we can get a dictonary with the english translation of the products

translation_dict = dict()
for i in range(len(portugese_prod)):
        translation_dict[portugese_prod[i]] = english_prod[i]

In [12]:
#now replace the portugese with english

product_info['product_category_name'] = product_info['product_category_name'].replace(translation_dict)

In [13]:
# in order to calculate the volume of the product and create new column need to check if there are any nans

In [14]:
product_info[product_info['product_length_cm'].isna()]['product_length_cm']

8578    NaN
18851   NaN
Name: product_length_cm, dtype: float64

In [15]:
product_info[product_info['product_height_cm'].isna()]['product_height_cm']

8578    NaN
18851   NaN
Name: product_height_cm, dtype: float64

In [16]:
product_info[product_info['product_width_cm'].isna()]['product_width_cm']

8578    NaN
18851   NaN
Name: product_width_cm, dtype: float64

In [17]:
#it looks like rows 8578 and 18851 are missing information so just have to ignore those columns to calculate volume

In [18]:
#I can ignore those rows with apply and a custom function

In [19]:
#writing function to calculate volume in cm cubed
def calc_prod_vol(df):
    #use try except block so that if I run into the product of nans I can replace the Nan with a 0 
    try:
        x = df['product_length_cm'] * df['product_height_cm'] * df['product_width_cm']
    except:
        x = 0
    return x

In [20]:
#use apply and function to create new column
product_info['volume_cm_cubed'] = product_info.apply(calc_prod_vol,axis = 1)

In [21]:
product_info

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,volume_cm_cubed
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumery,40.0,287.0,1.0,225.0,16.0,10.0,14.0,2240.0
1,3aa071139cb16b67ca9e5dea641aaa2f,art,44.0,276.0,1.0,1000.0,30.0,18.0,20.0,10800.0
2,96bd76ec8810374ed1b65e291975717f,sports_leisure,46.0,250.0,1.0,154.0,18.0,9.0,15.0,2430.0
3,cef67bcfe19066a932b7673e239eb23d,baby,27.0,261.0,1.0,371.0,26.0,4.0,26.0,2704.0
4,9dc1a7de274444849c219cff195d0b71,housewares,37.0,402.0,4.0,625.0,20.0,17.0,13.0,4420.0
...,...,...,...,...,...,...,...,...,...,...
32946,a0b7d5a992ccda646f2d34e418fff5a0,furniture_decor,45.0,67.0,2.0,12300.0,40.0,40.0,40.0,64000.0
32947,bf4538d88321d0fd4412a93c974510e6,construction_tools_lights,41.0,971.0,1.0,1700.0,16.0,19.0,16.0,4864.0
32948,9a7c6041fa9592d9d9ef6cfe62a71f8c,bed_bath_table,50.0,799.0,1.0,1400.0,27.0,7.0,27.0,5103.0
32949,83808703fc0706a22e264b9d75f04a2e,computers_accessories,60.0,156.0,2.0,700.0,31.0,13.0,20.0,8060.0


In [22]:
# going to translate the reviews from Portugese to English

In [ ]:
#importing the translating package so I can translate the portugese reviews
import translators as ts


In [ ]:
review_scores[~review_scores['review_comment_message'].isna()]
#there are 40977 comments that need to be translated

In [ ]:
#creating custom function to go line by line and translate comments
def translate_to_english(df):
    try:
        #translation part
        review = df['review_comment_message']
        line = ts.translate_text(review)
    except:
        #return empty string if there is no message
        line = ''
    return line

In [ ]:
#lets say each comment takes 2 seconds to translate.
40977* 2 / 60 / 60

#which is 22.765 hours - im going to have to break this up

In [ ]:
#reviews_1_5000 = review_scores[:5000].apply(translate_to_english,axis=1)

In [ ]:
#comment the code that sends reviews to a pickle so don't overwrite the pickle

#with open('reviews_1_5000.pickle', 'wb') as handle:
    #pickle.dump(reviews_1_5000, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#imports rows 0-4999 into notebook data
#with open('reviews_1_5000.pickle', 'rb') as handle:
    #reviews_1_5000 = pickle.load(handle)

In [ ]:
reviews_1_5000

In [ ]:
reviews_5000_10000 = review_scores[5000:10000].apply(translate_to_english,axis=1)

In [ ]:
reviews_5000_10000

In [ ]:
#with open('reviews_5000_10000.pickle', 'wb') as handle:
    #pickle.dump(reviews_5000_10000, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
reviews_10000_15000 = review_scores[10000:15000].apply(translate_to_english,axis=1)

In [ ]:
reviews_10000_15000

In [ ]:
#with open('reviews_10000_15000.pickle', 'wb') as handle:
    #pickle.dump(reviews_10000_15000, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
reviews_15000_20000 = review_scores[15000:20000].apply(translate_to_english,axis=1)

In [ ]:
reviews_15000_20000

In [ ]:
#with open('reviews_15000_20000.pickle', 'wb') as handle:
    #pickle.dump(reviews_15000_20000, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
reviews_20000_25000 = review_scores[20000:25000].apply(translate_to_english,axis=1)

In [ ]:
reviews_20000_25000

In [ ]:
#with open('reviews_20000_25000.pickle', 'wb') as handle:
    #pickle.dump(reviews_20000_25000, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
reviews_25000_30000 = review_scores[25000:30000].apply(translate_to_english,axis=1)

In [ ]:
reviews_25000_30000

In [ ]:
#with open('reviews_25000_30000.pickle', 'wb') as handle:
    #pickle.dump(reviews_25000_30000, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
reviews_30000_35000 = review_scores[30000:35000].apply(translate_to_english,axis=1)

In [ ]:
reviews_30000_35000

In [ ]:
#with open('reviews_30000_35000.pickle', 'wb') as handle:
    #pickle.dump(reviews_30000_35000, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
reviews_35000_40000 = review_scores[35000:40000].apply(translate_to_english,axis=1)
reviews_35000_40000

In [ ]:
#with open('reviews_35000_40000.pickle', 'wb') as handle:
    #pickle.dump(reviews_35000_40000, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
reviews_40000_45000 = review_scores[40000:45000].apply(translate_to_english,axis=1)
reviews_40000_45000

In [ ]:
#with open('reviews_40000_45000.pickle', 'wb') as handle:
    #pickle.dump(reviews_40000_45000, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
reviews_45000_50000 = review_scores[45000:50000].apply(translate_to_english,axis=1)
reviews_45000_50000

In [ ]:
#with open('reviews_45000_50000.pickle', 'wb') as handle:
    #pickle.dump(reviews_45000_50000, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
reviews_50000_55000 = review_scores[50000:55000].apply(translate_to_english,axis=1)
reviews_50000_55000

In [ ]:
#with open('reviews_50000_55000.pickle', 'wb') as handle:
    #pickle.dump(reviews_50000_55000, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
reviews_55000_60000 = review_scores[55000:60000].apply(translate_to_english,axis=1)
reviews_55000_60000

In [ ]:
#with open('reviews_55000_60000.pickle', 'wb') as handle:
    #pickle.dump(reviews_55000_60000, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
reviews_60000_65000 = review_scores[60000:65000].apply(translate_to_english,axis=1)
reviews_60000_65000

In [ ]:
#with open('Pickled Translated Reviews/reviews_60000_65000.pickle', 'wb') as handle:
    #pickle.dump(reviews_60000_65000, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
reviews_65000_70000 = review_scores[65000:70000].apply(translate_to_english,axis=1)
reviews_65000_70000

In [ ]:
#with open('Pickled Translated Reviews/reviews_65000_70000.pickle', 'wb') as handle:
    #pickle.dump(reviews_65000_70000, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
reviews_70000_75000 = review_scores[70000:75000].apply(translate_to_english,axis=1)
reviews_70000_75000

In [ ]:
#with open('Pickled Translated Reviews/reviews_70000_75000.pickle', 'wb') as handle:
    #pickle.dump(reviews_70000_75000, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
reviews_75000_80000 = review_scores[75000:80000].apply(translate_to_english,axis=1)
reviews_75000_80000

In [ ]:
#with open('Pickled Translated Reviews/reviews_75000_80000.pickle', 'wb') as handle:
    #pickle.dump(reviews_75000_80000, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
reviews_80000_85000 = review_scores[80000:85000].apply(translate_to_english,axis=1)
reviews_80000_85000

In [ ]:
#with open('Pickled Translated Reviews/reviews_80000_85000.pickle', 'wb') as handle:
    #pickle.dump(reviews_80000_85000, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
reviews_85000_90000 = review_scores[85000:90000].apply(translate_to_english,axis=1)
reviews_85000_90000

In [ ]:
#with open('Pickled Translated Reviews/reviews_85000_90000.pickle', 'wb') as handle:
    #pickle.dump(reviews_85000_90000, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
reviews_90000_95000 = review_scores[90000:95000].apply(translate_to_english,axis=1)
reviews_90000_95000

In [ ]:
#with open('Pickled Translated Reviews/reviews_90000_95000.pickle', 'wb') as handle:
    #pickle.dump(reviews_90000_95000, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
len(review_scores)

In [ ]:
reviews_95000_99224 = review_scores[95000:99225].apply(translate_to_english,axis=1)
reviews_95000_99224

In [ ]:
#with open('Pickled Translated Reviews/reviews_95000_99224.pickle', 'wb') as handle:
    #pickle.dump(reviews_95000_99224, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
lst1=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]
lst2=[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
lst1 = [x*5 for x in lst1]
lst2 = [x*5 for x in lst2]

In [ ]:
#start the dataframe with that will contain all reviews with the first 5000 reviews
with open('Pickled Translated Reviews/reviews_1_5000.pickle', 'rb') as handle:
        all_english_reviews = pickle.load(handle)
for i,j in zip(lst1,lst2):
    string = 'reviews_'+str(i)+'000_' + str(j)+'000'
    print(string)
    #retrieving data
    with open('Pickled Translated Reviews/'+string+'.pickle', 'rb') as handle:
        string = pickle.load(handle)
    #concatinating each set of reviews
    all_english_reviews = pd.concat([all_english_reviews, string], ignore_index=True,axis=0)
#concatinate last set of reviews
with open('Pickled Translated Reviews/reviews_95000_99224.pickle', 'rb') as handle:
    string = pickle.load(handle)   
all_english_reviews = pd.concat([all_english_reviews, string], ignore_index=True,axis=0)
all_english_reviews

In [ ]:
len(all_english_reviews.index)

In [ ]:
#with open('Pickled Translated Reviews/All reviews combined/all_english_reviews.pickle', 'wb') as handle:
    #pickle.dump(all_english_reviews, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('Pickled Translated Reviews/All reviews combined/all_english_reviews.pickle', 'rb') as handle:
    all_english_reviews = pickle.load(handle)

In [26]:
rows_w_comment=review_scores[~review_scores['review_comment_message'].isna()]
rows_w_comment

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53
9,8670d52e15e00043ae7de4c01cc2fe06,b9bf720beb4ab3728760088589c62129,4,recomendo,aparelho eficiente. no site a marca do aparelh...,2018-05-22 00:00:00,2018-05-23 16:45:47
12,4b49719c8a200003f700d3d986ea1a19,9d6f15f95d01e79bd1349cc208361f09,4,NaN,"Mas um pouco ,travando...pelo valor ta Boa.\r\n",2018-02-16 00:00:00,2018-02-20 10:52:22
15,3948b09f7c818e2d86c9a546758b2335,e51478e7e277a83743b6f9991dbfa3fb,5,Super recomendo,"Vendedor confiável, produto ok e entrega antes...",2018-05-23 00:00:00,2018-05-24 03:00:01
...,...,...,...,...,...,...,...
99205,98fffa80dc9acbde7388bef1600f3b15,d398e9c82363c12527f71801bf0e6100,4,NaN,para este produto recebi de acordo com a compr...,2017-11-29 00:00:00,2017-11-30 15:52:51
99208,df5fae90e85354241d5d64a8955b2b09,509b86c65fe4e2ad5b96408cfef9755e,5,NaN,Entregou dentro do prazo. O produto chegou em ...,2018-02-07 00:00:00,2018-02-19 19:47:23
99215,a709d176f59bc3af77f4149c96bae357,d5cb12269711bd1eaf7eed8fd32a7c95,3,NaN,"O produto não foi enviado com NF, não existe v...",2018-05-19 00:00:00,2018-05-20 21:51:06
99221,b3de70c89b1510c4cd3d0649fd302472,55d4004744368f5571d1f590031933e4,5,NaN,"Excelente mochila, entrega super rápida. Super...",2018-03-22 00:00:00,2018-03-23 09:10:43


In [ ]:
review_scores

In [23]:
from googletrans import Translator
translator = Translator()

In [24]:
#try to transalte one line
translator.translate('Mas um pouco ,travando...pelo valor ta Boa.').text

"But a little slow...for the price, it's good."

In [42]:
#create new dataframe with same index as row_w_comments
d = [''] * len(rows_w_comment)
df_english_reviews2 = pd.DataFrame(d,columns = ['english reviews'],index=rows_w_comment.index)
df_english_reviews2

#translate from portugese to english
for review, i in zip(rows_w_comment.loc[23181:,'review_comment_message'],rows_w_comment[rows_w_comment.index>=23181].index):
    t = translator.translate(review).text.strip('\n').strip('\r')
    time.sleep(1+random.random()*2)
    #print(review)
    #print(t)
    #print()
    df_english_reviews2.loc[i,'english reviews'] = t
    print(i)

23181
23182
23185
23187
23190
23191
23195
23196
23204
23206
23207
23208
23209
23213
23215
23216
23218
23221
23226
23231
23232
23233
23234
23235
23238
23241
23242
23243
23245
23246
23249
23250
23254
23255
23256
23258
23260
23262
23265
23268
23277
23280
23282
23290
23291
23292
23298
23301
23306
23307
23308
23310
23312
23316
23317
23318
23319
23322
23323
23329
23335
23337
23340
23345
23347
23349
23353
23357
23359
23361
23367
23371
23372
23374
23375
23377
23380
23381
23385
23387
23388
23389
23395
23400
23406
23409
23410
23412
23417
23419
23420
23422
23424
23426
23428
23432
23434
23435
23437
23440
23441
23443
23444
23445
23446
23447
23448
23449
23452
23454
23455
23459
23460
23461
23464
23465
23466
23467
23469
23470
23471
23473
23474
23475
23476
23477
23478
23480
23481
23482
23483
23485
23486
23489
23492
23494
23495
23496
23499
23500
23501
23503
23504
23509
23512
23513
23515
23518
23522
23524
23525
23527
23529
23530
23532
23534
23542
23544
23550
23555
23565
23566
23573
23576
23578
23584
2358

In [34]:
23179

,english reviews
3,I received it well before the stipulated deadl...
4,Congratulations lannister stores I loved shopp...
9,efficient device. On the website the brand of ...
12,"But a little slow...for the price, it's good."
15,"Reliable seller, ok product and delivery on time."
...,...
99205,
99208,
99215,
99221,


In [36]:
with open('df_english_reviews2.pickle', 'wb') as handle:
    pickle.dump(df_english_reviews2, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [41]:
rows_w_comment[rows_w_comment.index>=23181].index

Index([23181, 23182, 23185, 23187, 23190, 23191, 23195, 23196, 23204, 23206,
       ...
       99196, 99197, 99200, 99202, 99203, 99205, 99208, 99215, 99221, 99223],
      dtype='int64', length=31279)